In [1]:
# Sheet properties

In [2]:
%%javascript
IPython.OutputArea.prototype._should_scroll = function(lines) {
    return false;
}


<IPython.core.display.Javascript object>

In [3]:
# Imports

In [4]:
import sys
sys.path.append('/home/rcendre/classification')
import os
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID";
os.environ["CUDA_VISIBLE_DEVICES"]="1";  
from numpy import array
from sklearn.preprocessing import LabelEncoder
from toolbox.models.builtin import Applications
from toolbox.classification.common import IO, Tools, Folds
from toolbox.classification.parameters import Settings, Dermatology
from toolbox.transforms.labels import OrderedEncoder
from toolbox.views.common import Views, ViewsTools
import warnings
warnings.filterwarnings('ignore')

In [5]:
# Parameters

In [6]:
validation = 4
settings = Settings.get_default_dermatology()
max_iter = -1

In [7]:
# Inputs

In [8]:
inputs = Dermatology.images(modality='Microscopy', data_type='Full', use_unknown=False)
prediction_file = f'Prediction_FineTune.pickle'

In [9]:
# Transform groups
group_encoder = LabelEncoder().fit(array(inputs['ID_Patient'].tolist()))
Tools.transform(inputs, {'datum': 'ID_Patient'}, group_encoder, 'GroupEncode')
# Transform labels
label_encoder = OrderedEncoder().fit(['Normal', 'Benign', 'Malignant'])
Tools.transform(inputs, {'datum': 'Label'}, label_encoder, 'LabelEncode')
# Make folds
Folds.build_group_folds(inputs, {'datum': 'Datum', 'label_encode': 'LabelEncode', 'group': 'GroupEncode'}, validation)

In [10]:
# Additionnal

In [11]:
advanced = {'batch_size': 8,
            'epochs' : 15,
            'shuffle': True,
            'rotation_range': 45,
            'width_shift_range': 0.1,
            'height_shift_range': 0.1,
            'horizontal_flip': True,
            'vertical_flip': True,
            'fill_mode': 'wrap'}

In [12]:
# Models

In [13]:
models = []
models.append(('RNAvgAdvanced', 'ResNet', 'avg', advanced))
models.append(('RNMaxAdvanced', 'ResNet', 'max', advanced))

In [14]:
# Evaluation

In [15]:
for name, architecture, pooling, additionnal in models:

    model = Applications.get_fine_tuning(3, 0, -1, architecture=architecture, pooling=pooling, activation='softmax', additional=additionnal)

    # Start evaluation
    print(f'{name} performed...', end='\r')
    Tools.evaluate(inputs, {'datum': 'Datum', 'label_encode': 'LabelEncode'}, model, name)

    # Save
    IO.save(inputs, prediction_file)

Found 4073 images belonging to 3 classes.
Pre-training...
Instructions for updating:
Please use Model.fit, which supports generators.
  ...
    to  
  ['...']
Train for 510 steps
Epoch 1/15
510/510 [==============================] - 869s 2s/step - loss: 0.6988 - accuracy: 0.7078
Epoch 2/15
510/510 [==============================] - 863s 2s/step - loss: 0.5798 - accuracy: 0.7626
Epoch 3/15
510/510 [==============================] - 864s 2s/step - loss: 0.5616 - accuracy: 0.7704
Epoch 4/15
510/510 [==============================] - 864s 2s/step - loss: 0.5559 - accuracy: 0.7746
Epoch 5/15
510/510 [==============================] - 866s 2s/step - loss: 0.5248 - accuracy: 0.7918
Epoch 6/15
510/510 [==============================] - 869s 2s/step - loss: 0.5058 - accuracy: 0.7997
Epoch 7/15
510/510 [==============================] - 864s 2s/step - loss: 0.4953 - accuracy: 0.8112
Epoch 8/15
510/510 [==============================] - 869s 2s/step - loss: 0.4641 - accuracy: 0.8132
Epoch 9/15
51

510/510 [==============================] - 870s 2s/step - loss: 0.5156 - accuracy: 0.7935
Epoch 9/15
510/510 [==============================] - 870s 2s/step - loss: 0.5316 - accuracy: 0.7768
Epoch 10/15
510/510 [==============================] - 869s 2s/step - loss: 0.4956 - accuracy: 0.7979
Epoch 11/15
510/510 [==============================] - 875s 2s/step - loss: 0.4987 - accuracy: 0.7997
Epoch 12/15
510/510 [==============================] - 869s 2s/step - loss: 0.5035 - accuracy: 0.7982
Epoch 13/15
510/510 [==============================] - 867s 2s/step - loss: 0.4780 - accuracy: 0.8019
Epoch 14/15
510/510 [==============================] - 875s 2s/step - loss: 0.4794 - accuracy: 0.8043
Epoch 15/15
510/510 [==============================] - 871s 2s/step - loss: 0.4645 - accuracy: 0.8063
Final-training...
  ...
    to  
  ['...']
Train for 510 steps
Epoch 1/15
510/510 [==============================] - 884s 2s/step - loss: 0.4329 - accuracy: 0.8281
Epoch 2/15
510/510 [=============

510/510 [==============================] - 892s 2s/step - loss: 0.5300 - accuracy: 0.8723
Epoch 7/15
510/510 [==============================] - 885s 2s/step - loss: 0.3835 - accuracy: 0.8998
Epoch 8/15
510/510 [==============================] - 882s 2s/step - loss: 0.3902 - accuracy: 0.8927
Epoch 9/15
510/510 [==============================] - 885s 2s/step - loss: 0.3546 - accuracy: 0.9067
Epoch 10/15
510/510 [==============================] - 886s 2s/step - loss: 0.2982 - accuracy: 0.9123
Epoch 11/15
510/510 [==============================] - 884s 2s/step - loss: 0.3512 - accuracy: 0.9035
Epoch 12/15
510/510 [==============================] - 895s 2s/step - loss: 0.2501 - accuracy: 0.9251
Epoch 13/15
510/510 [==============================] - 902s 2s/step - loss: 0.2895 - accuracy: 0.9151
Epoch 14/15
510/510 [==============================] - 898s 2s/step - loss: 0.2113 - accuracy: 0.9354
Epoch 15/15
510/510 [==============================] - 901s 2s/step - loss: 0.2422 - accuracy: 0.

510/510 [==============================] - 912s 2s/step - loss: 4.4948 - accuracy: 0.6279
Epoch 4/15
510/510 [==============================] - 910s 2s/step - loss: 5.0217 - accuracy: 0.6330
Epoch 5/15
510/510 [==============================] - 908s 2s/step - loss: 4.2578 - accuracy: 0.6627
Epoch 6/15
510/510 [==============================] - 913s 2s/step - loss: 4.2823 - accuracy: 0.6708
Epoch 7/15
510/510 [==============================] - 903s 2s/step - loss: 4.1521 - accuracy: 0.6802
Epoch 8/15
510/510 [==============================] - 913s 2s/step - loss: 4.6071 - accuracy: 0.6745
Epoch 9/15
510/510 [==============================] - 933s 2s/step - loss: 4.4222 - accuracy: 0.6745
Epoch 10/15
510/510 [==============================] - 922s 2s/step - loss: 4.4892 - accuracy: 0.6861
Epoch 11/15
510/510 [==============================] - 913s 2s/step - loss: 4.4323 - accuracy: 0.6929
Epoch 12/15
510/510 [==============================] - 914s 2s/step - loss: 4.2515 - accuracy: 0.690

In [16]:
# Scores and ROC

In [17]:
from IPython.display import HTML
from IPython.display import display

inputs = IO.load(prediction_file)

# Transform labels
label_encoder = OrderedEncoder().fit(['Normal', 'Benign', 'Malignant'])

# ROC Curve
ViewsTools.plot_size((8,8))

for name, architecture, pooling, additionnal in models:

        # Label
        display(HTML(ViewsTools.dataframe_renderer([Views.report(inputs, {'label_encode': 'LabelEncode', 'eval': name}, label_encoder)],
                                                                        title=[f'Test - {name}'])))

,precision,recall,f1-score,support
Normal,0.20±0.10,0.80±0.06,0.31±0.13,757.00±115.20
Benign,0.67±0.26,0.12±0.05,0.21±0.09,2204.00±115.21
Malignant,0.79±0.10,0.61±0.13,0.69±0.06,2470.00±125.50
accuracy,0.44±0.06,0.44±0.06,0.44±0.06,0.44±0.06
macro avg,0.55±0.07,0.51±0.06,0.40±0.07,5431.00±0.43
weighted avg,0.66±0.12,0.44±0.06,0.44±0.05,5431.00±0.43


,precision,recall,f1-score,support
Normal,0.24±0.13,0.77±0.12,0.37±0.14,757.00±115.20
Benign,0.86±0.14,0.25±0.07,0.39±0.10,2204.00±115.21
Malignant,0.79±0.09,0.76±0.12,0.78±0.05,2470.00±125.50
accuracy,0.56±0.05,0.56±0.05,0.56±0.05,0.56±0.05
macro avg,0.63±0.05,0.60±0.04,0.51±0.06,5431.00±0.43
weighted avg,0.74±0.11,0.56±0.05,0.56±0.05,5431.00±0.43
